Treinar uma rede neural para prever o valor de um carro usado

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import pandas as pd

Lendo a base

In [3]:
base = pd.read_csv("autos.csv", encoding='ISO-8859-1')

In [4]:
base

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371523,2016-03-14 17:48:27,Suche_t4___vito_ab_6_sitze,privat,Angebot,2200,test,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,2016-03-14 00:00:00,0,39576,2016-04-06 00:46:52
371524,2016-03-05 19:56:21,Smart_smart_leistungssteigerung_100ps,privat,Angebot,1199,test,cabrio,2000,automatik,101,fortwo,125000,3,benzin,smart,nein,2016-03-05 00:00:00,0,26135,2016-03-11 18:17:12
371525,2016-03-19 18:57:12,Volkswagen_Multivan_T4_TDI_7DC_UY2,privat,Angebot,9200,test,bus,1996,manuell,102,transporter,150000,3,diesel,volkswagen,nein,2016-03-19 00:00:00,0,87439,2016-04-07 07:15:26
371526,2016-03-20 19:41:08,VW_Golf_Kombi_1_9l_TDI,privat,Angebot,3400,test,kombi,2002,manuell,100,golf,150000,6,diesel,volkswagen,NaN,2016-03-20 00:00:00,0,40764,2016-03-24 12:45:21


A base esta bagunçada, com um mesmo modelo escrito de varias formas, colunas não relevantes e valores nulos 

In [5]:
base.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'kilometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [6]:
base.drop(['dateCrawled', 'dateCreated', 'nrOfPictures', 'postalCode', 'lastSeen'], axis=1, inplace=True)

In [7]:
base.columns

Index(['name', 'seller', 'offerType', 'price', 'abtest', 'vehicleType',
       'yearOfRegistration', 'gearbox', 'powerPS', 'model', 'kilometer',
       'monthOfRegistration', 'fuelType', 'brand', 'notRepairedDamage'],
      dtype='object')

In [8]:
base['name'].value_counts()

name
Ford_Fiesta                                              657
BMW_318i                                                 627
Opel_Corsa                                               622
Volkswagen_Golf_1.4                                      603
BMW_316i                                                 523
                                                        ... 
Audi_A4_Avant_Klima_Gruene_Plakette_TÜV_&AU_NEU_XENON      1
Renault_clio_in_gold_450VB_!!                              1
Fiat_Doblo_1.6_Multijet                                    1
Renault_Laguna_1                                           1
BMW_M135i_vollausgestattet_NP_52.720____Euro               1
Name: count, Length: 233531, dtype: int64

In [9]:
base['seller'].value_counts()

seller
privat        371525
gewerblich         3
Name: count, dtype: int64

In [10]:
base['offerType'].value_counts()

offerType
Angebot    371516
Gesuch         12
Name: count, dtype: int64

In [11]:
base.drop(['name','seller', 'offerType'], axis=1, inplace=True) # Eliminando colunas com dados desbalanceados

In [12]:
base.columns

Index(['price', 'abtest', 'vehicleType', 'yearOfRegistration', 'gearbox',
       'powerPS', 'model', 'kilometer', 'monthOfRegistration', 'fuelType',
       'brand', 'notRepairedDamage'],
      dtype='object')

Eliminando registros com o preço muito alto ou muito baixo

In [13]:
base.loc[base['price'] <= 10] # nao faz sentido o preço ser menor que 10 euros

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
7,0,test,limousine,1980,manuell,50,andere,40000,7,benzin,volkswagen,nein
40,0,test,NaN,1990,NaN,0,corsa,150000,1,benzin,opel,NaN
60,1,control,suv,1994,manuell,286,NaN,150000,11,NaN,sonstige_autos,NaN
91,1,control,limousine,1995,manuell,113,e_klasse,150000,4,diesel,mercedes_benz,nein
115,0,test,NaN,2017,manuell,0,golf,5000,12,benzin,volkswagen,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371356,0,control,NaN,2000,manuell,65,corsa,150000,0,NaN,opel,ja
371392,0,test,kleinwagen,2002,manuell,60,fiesta,150000,3,benzin,ford,NaN
371402,0,control,kleinwagen,1999,manuell,53,swift,150000,3,benzin,suzuki,NaN
371431,0,control,kleinwagen,1999,manuell,37,arosa,150000,7,benzin,seat,ja


In [14]:
base.loc[base['price'] > 350000]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
1846,579000,control,coupe,1980,manuell,277,andere,20000,12,benzin,bmw,nein
10649,420000,control,coupe,2004,manuell,483,911,50000,4,benzin,porsche,nein
14663,11111111,control,coupe,2003,manuell,64,polo,150000,2,benzin,volkswagen,NaN
16889,1000000,control,kombi,1998,NaN,0,mondeo,150000,0,benzin,ford,ja
20143,1250000,test,coupe,2016,manuell,500,911,5000,3,benzin,porsche,nein
...,...,...,...,...,...,...,...,...,...,...,...,...
364171,3890000,test,coupe,2006,NaN,799,NaN,5000,7,NaN,sonstige_autos,nein
365461,599000,control,coupe,1980,manuell,377,andere,5000,3,benzin,bmw,nein
366653,99999999,control,cabrio,1996,manuell,192,3er,150000,0,NaN,bmw,NaN
366861,3895000,test,coupe,2006,NaN,799,NaN,5000,4,benzin,sonstige_autos,nein


In [15]:
base = base[(base['price'] > 10) | (base['price'] > 350000)]

In [16]:
base.shape

(359410, 12)

Tratamento de nulos

In [17]:
pd.isnull(base['vehicleType']) # serie booleana onde o tipo do veiculo esta nulo

0          True
1         False
2         False
3         False
4         False
          ...  
371523     True
371524    False
371525    False
371526    False
371527    False
Name: vehicleType, Length: 359410, dtype: bool

In [18]:
base.loc[pd.isnull(base['vehicleType'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN
16,300,test,NaN,2016,NaN,60,polo,150000,0,benzin,volkswagen,NaN
22,2900,test,NaN,2018,manuell,90,meriva,150000,5,benzin,opel,nein
26,5555,control,NaN,2017,manuell,125,c4,125000,4,NaN,citroen,nein
31,899,control,NaN,2016,manuell,60,clio,150000,6,benzin,renault,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371495,180,control,NaN,1995,NaN,0,NaN,125000,3,benzin,opel,NaN
371504,2600,control,NaN,2005,automatik,0,c_klasse,150000,9,NaN,mercedes_benz,NaN
371509,1900,test,NaN,2000,manuell,110,NaN,150000,7,NaN,volkswagen,nein
371519,5250,control,NaN,2016,automatik,150,159,150000,12,NaN,alfa_romeo,nein


In [19]:
base['vehicleType'].value_counts()

vehicleType
limousine     93627
kleinwagen    78019
kombi         65931
bus           29702
cabrio        22519
coupe         18425
suv           14479
andere         3129
Name: count, dtype: int64

In [20]:
base.loc[pd.isnull(base['gearbox'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
15,450,test,kleinwagen,1910,NaN,0,ka,5000,0,benzin,ford,NaN
16,300,test,NaN,2016,NaN,60,polo,150000,0,benzin,volkswagen,NaN
32,245,test,limousine,1994,NaN,0,golf,150000,2,benzin,volkswagen,nein
37,1500,test,NaN,2016,NaN,0,kangoo,150000,1,diesel,renault,nein
70,1200,test,coupe,2001,NaN,0,astra,150000,0,NaN,opel,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371443,3300,control,kombi,2006,NaN,0,touran,150000,7,diesel,volkswagen,NaN
371460,3500,control,NaN,1995,NaN,0,polo,150000,0,NaN,volkswagen,NaN
371486,350,control,kleinwagen,1996,NaN,65,punto,150000,0,NaN,fiat,NaN
371495,180,control,NaN,1995,NaN,0,NaN,125000,3,benzin,opel,NaN


In [21]:
base['gearbox'].value_counts()

gearbox
manuell      266612
automatik     75522
Name: count, dtype: int64

In [22]:
base.loc[pd.isnull(base['model'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
1,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja
83,350,control,kleinwagen,1997,manuell,54,NaN,150000,3,NaN,fiat,ja
139,1450,control,limousine,1992,manuell,136,NaN,150000,0,NaN,audi,nein
156,6799,control,kleinwagen,2009,NaN,60,NaN,20000,5,benzin,volkswagen,nein
165,500,control,kleinwagen,1999,manuell,0,NaN,150000,0,benzin,renault,nein
...,...,...,...,...,...,...,...,...,...,...,...,...
371399,560,control,kleinwagen,2001,automatik,170,NaN,90000,0,benzin,fiat,ja
371476,9400,control,kombi,2007,manuell,200,NaN,150000,4,diesel,sonstige_autos,ja
371495,180,control,NaN,1995,NaN,0,NaN,125000,3,benzin,opel,NaN
371509,1900,test,NaN,2000,manuell,110,NaN,150000,7,NaN,volkswagen,nein


In [23]:
base['model'].value_counts()

model
golf               28998
andere             25571
3er                19907
polo               12608
corsa              12149
                   ...  
serie_2                8
rangerover             6
serie_3                3
serie_1                1
discovery_sport        1
Name: count, Length: 251, dtype: int64

In [24]:
base.loc[pd.isnull(base['fuelType'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
9,999,test,kleinwagen,1998,manuell,101,golf,150000,0,NaN,volkswagen,NaN
13,2500,control,kombi,2004,manuell,131,passat,150000,2,NaN,volkswagen,nein
26,5555,control,NaN,2017,manuell,125,c4,125000,4,NaN,citroen,nein
36,1600,control,andere,1991,manuell,75,kadett,70000,0,NaN,opel,NaN
41,7500,control,limousine,2002,automatik,306,e_klasse,150000,4,NaN,mercedes_benz,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371496,3850,test,cabrio,2006,manuell,108,2_reihe,125000,2,NaN,peugeot,nein
371504,2600,control,NaN,2005,automatik,0,c_klasse,150000,9,NaN,mercedes_benz,NaN
371509,1900,test,NaN,2000,manuell,110,NaN,150000,7,NaN,volkswagen,nein
371519,5250,control,NaN,2016,automatik,150,159,150000,12,NaN,alfa_romeo,nein


In [25]:
base['fuelType'].value_counts()

fuelType
benzin     217648
diesel     106013
lpg          5223
cng           557
hybrid        271
andere        166
elektro       101
Name: count, dtype: int64

In [26]:
base.loc[pd.isnull(base['notRepairedDamage'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN
2,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN
8,14500,control,bus,2014,manuell,125,c_max,30000,8,benzin,ford,NaN
9,999,test,kleinwagen,1998,manuell,101,golf,150000,0,NaN,volkswagen,NaN
12,999,control,kombi,1995,manuell,115,passat,150000,11,benzin,volkswagen,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371507,5999,test,kombi,2005,manuell,140,a4,150000,4,diesel,audi,NaN
371514,999,control,cabrio,2000,manuell,95,megane,150000,4,benzin,renault,NaN
371515,1690,test,kombi,2004,manuell,55,fabia,150000,4,benzin,skoda,NaN
371523,2200,test,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN


In [27]:
base['notRepairedDamage'].value_counts()

notRepairedDamage
nein    259366
ja       34014
Name: count, dtype: int64

In [28]:
valores = {
    'vehicleType'       : 'limousine',
    'gearbox'           : 'manuell',
    'model'             : 'golf',
    'fuelType'          : 'benzin',
    'notRepairedDamage' : 'nein'
}

base = base.fillna(value=valores)

In [29]:
base.isnull().sum()

price                  0
abtest                 0
vehicleType            0
yearOfRegistration     0
gearbox                0
powerPS                0
model                  0
kilometer              0
monthOfRegistration    0
fuelType               0
brand                  0
notRepairedDamage      0
dtype: int64

Após tratar a base, criar features e labels

In [30]:
base.columns

Index(['price', 'abtest', 'vehicleType', 'yearOfRegistration', 'gearbox',
       'powerPS', 'model', 'kilometer', 'monthOfRegistration', 'fuelType',
       'brand', 'notRepairedDamage'],
      dtype='object')

In [31]:
# queremos determinar o preço a partir das outras colunas
X = base.iloc[:, 1:].values # features
y = base.iloc[:,0].values # labels

In [32]:
base.brand.value_counts()

brand
volkswagen        76778
bmw               38996
opel              38561
mercedes_benz     34506
audi              31892
ford              24766
renault           17367
peugeot           10777
fiat               9341
seat               6815
skoda              5572
mazda              5530
smart              5159
citroen            5039
nissan             4893
toyota             4616
hyundai            3585
sonstige_autos     3357
mini               3354
volvo              3230
mitsubishi         2956
honda              2726
kia                2494
suzuki             2275
alfa_romeo         2251
porsche            2174
chevrolet          1792
chrysler           1373
dacia               889
jeep                785
daihatsu            774
land_rover          759
subaru              746
jaguar              604
daewoo              530
saab                516
trabant             490
lancia              467
rover               462
lada                213
Name: count, dtype: int64

Agora temos que tratar os atributos categoricos, porém apenas mapear as marcas para numeros pode dar a ideia de atributos categoricos hierarquizados, já que a rede neural utiliza pesos e valores, então faremos:

0 0 0 0 1  
0 0 0 1 0  
0 0 1 0 0  
. . .

In [33]:
base

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,limousine,1993,manuell,0,golf,150000,0,benzin,volkswagen,nein
1,18300,test,coupe,2011,manuell,190,golf,125000,5,diesel,audi,ja
2,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,nein
3,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein
4,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein
...,...,...,...,...,...,...,...,...,...,...,...,...
371523,2200,test,limousine,2005,manuell,0,golf,20000,1,benzin,sonstige_autos,nein
371524,1199,test,cabrio,2000,automatik,101,fortwo,125000,3,benzin,smart,nein
371525,9200,test,bus,1996,manuell,102,transporter,150000,3,diesel,volkswagen,nein
371526,3400,test,kombi,2002,manuell,100,golf,150000,6,diesel,volkswagen,nein


In [34]:
ohe = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [0,1,3,5,8,9,10])], remainder='passthrough')

In [35]:
X = ohe.fit_transform(X).toarray()

Para cada categoria de uma coluna são criadas 2^numeroCategorias colunas 

In [36]:
X.shape

(359410, 316)

#### Criando a rede neural

In [37]:
regressor = Sequential([
    tf.keras.layers.InputLayer(shape=(316,)),
    tf.keras.layers.Dense(units = int((316+1)/2), activation='relu'),
    tf.keras.layers.Dense(units = int((316+1)/2), activation='relu'),
    tf.keras.layers.Dense(units = 1, activation='linear')
])

In [38]:
regressor.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 158)            │        50,086 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 158)            │        25,122 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           159 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 75,367 (294.40 KB)

 Trainable params: 75,367 (294.40 KB)

 Non-trainable params: 0 (0.00 B)

In [39]:
regressor.compile(loss='mean_absolute_error',  optimizer='adam', metrics=['mean_absolute_error']) #loss é em modulo


In [40]:
regressor.fit(X, y, batch_size=300, epochs=100)

Epoch 1/100
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 11361.0742 - mean_absolute_error: 11361.0742
Epoch 2/100
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 14617.9453 - mean_absolute_error: 14617.9453
Epoch 3/100
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 23643.0391 - mean_absolute_error: 23643.0391
Epoch 4/100
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 16305.6660 - mean_absolute_error: 16305.6660
Epoch 5/100
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 8508.8379 - mean_absolute_error: 8508.8389
Epoch 6/100
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 10183.5791 - mean_absolute_error: 10183.5791
Epoch 7/100
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 8177.2466 - mean_absolute_error: 8177.2466
Epoch 8/100
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 9833.6777 - mean_absolute_error: 9833.6777
Epoch 9/100
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 9872.8428 - mean_absolute_error: 9872.8428
Epoch 10/100
1199/1199 ━━━━━━━━━━━━━

In [41]:
previsoes = regressor.predict(X)

11232/11232 ━━━━━━━━━━━━━━━━━━━━ 9s 756us/step


In [42]:
previsoes

array([[  576.1633],
       [ 8578.553 ],
       [10387.2295],
       ...,
       [ 4196.051 ],
       [ 1898.3749],
       [26545.55  ]], dtype=float32)

In [43]:
y

array([  480, 18300,  9800, ...,  9200,  3400, 28990], dtype=int64)

#### cross validation

In [44]:
import time
from scikeras.wrappers import KerasRegressor
from tensorflow.keras import backend as k
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn import metrics

In [45]:
inicio = time.time()

In [46]:
def criar_rede():
    k.clear_session()
    regressor = Sequential([
        tf.keras.layers.InputLayer(shape=(316,)),
        tf.keras.layers.Dense(units=158, activation='relu'),
        tf.keras.layers.Dense(units=158, activation='relu'),
        tf.keras.layers.Dense(units=1, activation='linear')])
    regressor.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
    return regressor

In [47]:
regressor = KerasRegressor(model = criar_rede, epochs = 100, batch_size = 300)

In [48]:
resultados = cross_val_score(estimator = regressor, X = X, y = y,
                             cv = 5, scoring = 'neg_mean_absolute_error')

#parei por tempo


Epoch 1/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 15259.7256 - mean_absolute_error: 15259.7256
Epoch 2/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 22046.5137 - mean_absolute_error: 22046.5117
Epoch 3/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 19205.3770 - mean_absolute_error: 19205.3770
Epoch 4/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 28108.9102 - mean_absolute_error: 28108.9121
Epoch 5/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 29946.2266 - mean_absolute_error: 29946.2266
Epoch 6/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 10489.5000 - mean_absolute_error: 10489.5000
Epoch 7/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 19107.6367 - mean_absolute_error: 19107.6367
Epoch 8/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 19003.3477 - mean_absolute_error: 19003.3477
Epoch 9/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 9177.4209 - mean_absolute_error: 9177.4209
Epoch 10/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3m